# 교차 검증과 그리드 서치

In [1]:
## 검증 세트를 바탕으로 하이퍼 파라미터 튜닝
##

In [2]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine-date')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

In [13]:
#기본은 5 fold

In [14]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00700116, 0.00599766, 0.00699329, 0.00798345, 0.0059967 ]), 'score_time': array([0.       , 0.0010035, 0.0010035, 0.       , 0.       ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [17]:
#5개 score점수 평균 취함

In [18]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
#dt 매개변수 변화시키면서 좋은 점수 나온것 찾음

# 분할기를 사용한 교차 검증

In [ ]:
#cv 폴드 개수
#회귀 kFlod
#분류 StratifiedKFold 

In [19]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [20]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#splitter 만든것 사용

scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [22]:
#딥러닝에서는 검증 세트가 많기 때문에 교차 검증 주로 사용하지 않음
#그리고 여러번 훈련하는 것이 비효율적

## 하이퍼파라미터 튜닝

In [23]:
from sklearn.model_selection import GridSearchCV
#어떤 매개 변수에서 좋은 성능을 보일지 확인
#매개변수를 딕셔너리로 저장
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
#두번째 매개변수에 만든 딕셔너리 사용

In [24]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
#n_jobs 사용정도 ,-1은 전부 사용

In [25]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [26]:
dt = gs.best_estimator_
#가장 좋은 파라미터 저장
print(dt.score(train_input, train_target))

0.9615162593804117


In [27]:
print(gs.best_params_)
#gs.best_params_값

{'min_impurity_decrease': 0.0001}


In [ ]:
#교차검증 결과

In [28]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [29]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [30]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [31]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [32]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [33]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [35]:
from scipy.stats import uniform, randint

In [36]:
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 3, 0, 1, 0, 9, 8, 7, 0, 5])

In [37]:
np.unique(rgen.rvs(1000), return_counts=True)
#각 숫자의 나온 개수

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98,  98,  98, 115,  99, 111,  82, 100, 101,  98], dtype=int64))

In [38]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.26798953, 0.61484498, 0.00416159, 0.16933508, 0.21478333,
       0.80591024, 0.6156819 , 0.85882763, 0.80626827, 0.68440643])

In [39]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [40]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
#100번 반복
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000167AB8C8C18>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000167AB8C8EF0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000167ACA0A1D0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000167ACA0A048>},
                   random_state=42)

In [41]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [42]:
print(np.max(gs.cv_results_['mean_test_score']))
#최적 파라미터의 성능

0.8695428296438884


In [43]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [44]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000167AB8C8C18>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000167AB8C8EF0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000167ACA0A1D0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000167ACA0A048>},
                   random_state=42)

In [45]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
